# Black-Box Room Modelling
You have a dataset collected from a room, which includes (i) indoor air temperature (Ti) in °C, (ii) outdoor air temperature (To) in °C, (iii) HVAC heating rate (q_HVAC) in W, and (iv) absorbed solar radiation (q_solar) in W. With this dataset, estimate the A and B matrixes.

### Load Libraries & Packages

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.signal import cont2discrete
import pickle
import matplotlib.pyplot as plt
import pygad

### Import and visualize Data

In [ ]:
with open('HW1_Data.pickle', 'rb') as f:
    Data = pickle.load(f)
Ti = Data['Ti'] # indoor air temperature in degC
To = Data['To'] # outdoor air temperature in degC
q_HVAC = Data['q_HVAC'] # HVAC heating rate in W
q_solar = Data['q_solar'] # absorbed solar radiation in W
N = To.shape[0] # number of timesteps

t_span = np.arange(0,N+1)
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].plot(Ti, label='T_indoor')
ax[0].plot(To, label='T_outdoor')
ax[0].set_xticks(t_span[::6*24],t_span[::6*24]*10/60)
ax[0].set_ylabel('Temperature [$\degree$C]')
ax[0].set_xlabel('Time [hour]')
ax[0].legend()

ax[1].plot(q_HVAC, label='q_HVAC')
ax[1].plot(q_solar, label='q_solar')
ax[1].set_xticks(t_span[::6*24],t_span[::6*24]*10/60)
ax[1].set_ylabel('Energy Rate [W]')
ax[1].set_xlabel('Time [hour]')
ax[1].legend()

plt.show()

In [ ]:
def simulation(params): 
    ##########################################################################################################
    # Please know that you don't have Rs and Cs any longer. You directly estimate the 10 elements in A and B #
    ##########################################################################################################
    
    # continuous time invariant state-space
    A = np.array([[params[0],params[1]],
                  [params[2],params[3]]])
    B = np.array([[params[4],params[5],params[6]],
                  [params[7],params[8],params[9]]])
    C = np.eye(2) # assume that C is known
    D = np.zeros((2,3)) # assume that D is known
      
    # discrete state-space
    Ad, Bd, Cd, Dd, _ = cont2discrete((A,B,C,D), 10*60) # discretization with 10 minute time interval
    
    x = np.zeros((2, N+1))
    x[0,0] = 20 # initial Ti
    x[1,0] = 15 # initial Tw
    
    # compute states over the simulation period
    for i in range(N):
        q_HVAC_t = q_HVAC[i]
        q_solar_t = q_solar[i]
        To_t = To[i]
        u_t = np.array([q_HVAC_t, q_solar_t, To_t])[:,None] # input vector
        x[:,i+1:i+2] = np.dot(Ad,x[:,i:i+1]) + np.dot(Bd,u_t)
    return x

def objective(ga_instance, solution, solution_idx):
    # run simulation
    x = simulation(solution)
    
    # error metric
    metric = ######################################## the same function you used for grey-box ########################################
    return 1/metric

In [ ]:
# parameter estimation with a genetic algorithm
last_fitness = 0
def on_generation(ga_instance):
    global last_fitness
    print(f"Generation = {ga_instance.generations_completed}")
    print(f"Fitness    = {ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]}")
    print(f"Change     = {ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1] - last_fitness}")
    last_fitness = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]

lb_params = [] # <------ lower bounds for the 10 elements in A and B. Add 10 values with comma seperators.
ub_params = [] # <------ upper bounds for the 10 elements in A and B. Add 10 values with comma seperators.

### for example,
# lb_params = [-6e-05, 4e-05, ..., 5e-06]
# ub_params = [-4e-05, 6e-05, ..., 7e-06]
# Where can we get resonable values for the bounds?

ga_instance = pygad.GA(num_generations=500,
                       num_parents_mating=10,
                       sol_per_pop=50,
                       gene_space=[{'low': lb_params[0], 'high': ub_params[0]},
                                   {'low': lb_params[1], 'high': ub_params[1]},
                                   {'low': lb_params[2], 'high': ub_params[2]},
                                   {'low': lb_params[3], 'high': ub_params[3]},
                                   {'low': lb_params[4], 'high': ub_params[4]},
                                   {'low': lb_params[5], 'high': ub_params[5]},
                                   {'low': lb_params[6], 'high': ub_params[6]},
                                   {'low': lb_params[7], 'high': ub_params[7]},
                                   {'low': lb_params[8], 'high': ub_params[8]},
                                   {'low': lb_params[9], 'high': ub_params[9]}],
                       num_genes=10, # <------------------------------------- the number of model parameters (the 10 elements in A and B)
                       fitness_func=objective,
                       on_generation=on_generation
                      )
ga_instance.run()
ga_instance.plot_fitness()
solution, solution_fitness, solution_idx = ga_instance.best_solution(ga_instance.last_generation_fitness)

In [ ]:
# scale them back
x = simulation(solution)

Ti_simulation = x[0,:] # Indoor temperature 
Tw_simulation = x[1,:]

# plotting
t_span = np.arange(0,N+1)
fig, ax = plt.subplots(1,2,figsize=(15,5))
ax[0].plot(Ti, label='Ti_original')
ax[0].plot(Ti_simulation, label='Ti_simulation', linestyle='--')
ax[0].plot(Tw_simulation, label='Tw_simulation')
ax[0].plot(To, label='To')
ax[0].set_xticks(t_span[::6*24],t_span[::6*24]*10/60)
ax[0].set_ylabel('Temperature [$\degree$C]')
ax[0].set_xlabel('Time [hour]')
ax[0].legend()

ax[1].plot(q_HVAC, label='q_HVAC')
ax[1].plot(q_solar, label='q_solar')
ax[1].set_xticks(t_span[::6*24],t_span[::6*24]*10/60)
ax[1].set_ylabel('Energy Rate [W]')
ax[1].set_xlabel('Time [hour]')
ax[1].legend()

plt.show()